##  Talent Scheduling 

In [1]:
#IMPORTs
import numpy as np
import matplotlib.pyplot as plt

#numba
from numba import njit, jit
from numba.typed import List

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

In [71]:
class ts_udp:
    """
    UDP (User-Defined Problem) class for the Talent Scheduling problem of the 
    from the course AuD(Team02[LG,DSc,RK,DSw])

    Summary
    ========
    The talent scheduling problem can be described as follows. A film producer needs 
    to schedule the scenes of his/her movie on a given location. Each scene has a duration (the days it takes to shoot it) 
    and requires some subset of the cast to be on location. The cast are paid for each day they are required to be on 
    location from the day the first scene they are in is shot, to the day the last scene they are in is shot, even though 
    some of those days they might not be required by the scene currently being shot (i.e., they will be on location waiting
    for the next scene they are in to be shot). Each cast member has a different daily salary. The aim of the film producer
    is to order the scenes in such a way as to minimize the salary cost of the shooting. 

    """

    def __init__(self, file_path):
        self.file_path = file_path
        self.name = ""
        self.num_scenes = 0
        self.num_actors = 0
        self.scenes = []
        self.actor_apprs = []
        self.actor_cost = []
        self.scene_duration = []
        self._load_instance()

    def _load_instance(self):
        """
        Load the instance from the dt-file.
        """
        with open(self.file_path, 'r') as file:
            lines = file.readlines()
            self.name = lines[0].strip()  # First line is the name of the data file
            self.num_scenes = int(lines[1].strip())  # Second line is the number of scenes
            self.num_actors = int(lines[2].strip())  # Third line is the number of actors
            
            # Reading actor data
            actor_data = lines[4:4 + self.num_actors]
            for actor_line in actor_data:
                parts = actor_line.strip().split()
                appearance = list(map(int, parts[:-1]))  # convert each part except last to int
                cost = int(parts[-1])  # Last part is the cost
                self.actor_apprs.append(appearance)
                self.actor_cost.append(cost)

            # Reading scene duration
            self.scene_duration = list(map(int, lines[5 + self.num_actors].strip().split()))



    ################
    # Public Methods
    ################

    def info(self):
        print(f"File name: {self.name}")
        print(f"Number of scenes: {self.num_scenes}, Number of actors: {self.num_actors}")
        print(f"Actor appearances and cost: {self.actor_apprs}")
        print(f"Actor appearances cost: {self.actor_cost}")
        print(f"Scene durations: {self.scene_duration}")

    #@njit
    def scene_cost(self):
        # Konvertieren der Liste in ein numpy 2D-Array, für Recheneffizienz
        array_actor_apprs = np.array(self.actor_apprs)
        
        # Array, mit dem multipliziert werden soll
        array_actor_cost = np.array(self.actor_cost)
        
        # Durchführen der Multiplikation mittels Broadcasting für effizienz
        zwischen_ergebnis = array_actor_apprs * array_actor_cost[:, np.newaxis]
        
        array_scene_duration = np.array(self.scene_duration)
        
        # Durchführen der zweiten Multiplikation mittels Broadcasting
        ergebnis = zwischen_ergebnis * array_scene_duration
        
        #print("Ergebnis mit Broadcasting:", ergebnis)
        
        sk=0
        for e in ergebnis:
            sk += e
        print(f'die Szenen-Kosten {sk}')
    
    #@njit
    #def ts_greedy(self, ):


    #@njit
    #def ts_rolout_greedy(self,)


In [72]:
film105 = ts_udp(file_path = 'data_prj/film105.dat')
film105.info()
film105.scene_cost()

File name: film105.dat
Number of scenes: 18, Number of actors: 8
Actor appearances and cost: [[0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1], [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1], [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1]]
Actor appearances cost: [10, 4, 5, 5, 5, 40, 4, 20]
Scene durations: [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]
die Szenen-Kosten [ 16   4  45  75  15  59  29  14  50  65  13  34  63  25 110  44   9  69]


### Instanzen 'small & small02'

```
small
6
4

1 0 0 0 1 1   1
1 1 0 1 0 0   1
0 1 1 0 1 0   1
0 0 1 1 0 1   1

1 2 3 4 5 6

============================================

small2
6
4

1 1 0 0 1 1   1
1 1 0 1 0 0   1
0 1 1 1 1 0   1
0 0 1 1 0 1   1

1 2 3 4 5 6
```

```
========================================== small
Solution:    3    1    0    2    4    5
Solution:    3    1    0    4    2    5
Solution:    3    1    2    0    4    5
Solution:    3    2    1    0    4    5
Solution:    4    2    1    0    3    5
Solution:    4    2    1    3    0    5
#Solutions=6
mincost = 54, rawcost = 42, extra = 12, bound = 134546788
ccc = 63,loop = 192
reuse = 124, reenter = 0, definite = 0, lookahead = 0, diffuse = 0, clump = 0
FINAL
  5   4   2   0   1   3 
------------------------
  X   X   -   X   .   . |  1
  .   .   .   X   X   X |  1
  .   X   X   -   X   . |  1
  X   -   X   -   -   X |  1
------------------------
  6   5   3   1   2   4 
------------------------
 12  15   9   4   6   8  = 54
------------------------
 12  10   6   2   4   8 
TIME = 0 ms
FINALDATA 6 4 6 54 63 124 0 0 0 tv small


========================================== small2
Solution:    4    0    1    3    2    5
Solution:    4    1    0    3    2    5
#Solutions=2
mincost = 56, rawcost = 48, extra = 8, bound = 134546788
ccc = 63,loop = 192
reuse = 124, reenter = 0, definite = 0, lookahead = 0, diffuse = 0, clump = 0
FINAL
  5   2   3   1   0   4 
------------------------
  X   -   -   X   X   X |  1
  .   .   X   X   X   . |  1
  .   X   X   X   -   X |  1
  X   X   X   .   .   . |  1
------------------------
  6   3   4   2   1   5 
------------------------
 12   9  16   6   3  10  = 56
------------------------
 12   6  12   6   2  10 
TIME = 0 ms
FINALDATA 6 4 6 56 63 124 0 0 0 tv small2